# Viser + MPlib Demo: Robot Visualization and IK/FK

This notebook demonstrates:
- Setting up a Viser visualization server
- Loading and visualizing robots from URDF files
- Using MPlib for inverse kinematics (IK) and forward kinematics (FK)
- Visualizing robot trajectories
- Helper functions for IK/FK calculations


## 1. Setup and Imports


In [49]:
import numpy as np
import time
from pathlib import Path

import viser
from viser.extras import ViserUrdf
import mplib

# Initialize the Viser server (opens a web interface at http://localhost:8080)
server = viser.ViserServer()


╭────── viser (listening *:8081) ───────╮
│             ╷                         │
│   HTTP      │ http://localhost:8081   │
│   Websocket │ ws://localhost:8081     │
│             ╵                         │
╰───────────────────────────────────────╯

## 2. Scene Setup

Add a world frame and configure the visualization scene.


In [50]:
# Add world frame for reference
server.scene.add_frame(
    "/world",
    wxyz=(1.0, 0.0, 0.0, 0.0),  # Quaternion (w, x, y, z) - identity rotation
    position=(0, 0, 0),
    axes_length=0.2,
    axes_radius=0.01,
)


FrameHandle(show_axes=True, axes_length=0.2, axes_radius=0.01, origin_radius=0.02, origin_color=(236, 236, 0))

## 3. Load Robot Models

Load the robot URDF and create visualization instances. Here we set up two robot instances side-by-side.


In [51]:
# Path to URDF file (adjust if needed)
URDF_PATH = Path("wxai_follower.urdf")
SRDF_PATH = Path("wxai_follower.srdf")  # Optional: for collision checking

# Create base frames for two robots side-by-side
Y_OFFSET = 0.28  # Separation between robots
base_a = server.scene.add_frame("/robots/a_base", position=(0.0, +Y_OFFSET, 0.0), show_axes=False)
base_b = server.scene.add_frame("/robots/b_base", position=(0.0, -Y_OFFSET, 0.0), show_axes=False)

# Load robot A
robot_a = ViserUrdf(
    server,
    urdf_or_path=URDF_PATH,
    load_meshes=True,
    load_collision_meshes=True,
    collision_mesh_color_override=(0.2, 0.2, 0.2, 0.5),  # Semi-transparent gray
    root_node_name="/robots/a_base",
)

# Load robot B
robot_b = ViserUrdf(
    server,
    urdf_or_path=URDF_PATH,
    load_meshes=True,
    load_collision_meshes=True,
    collision_mesh_color_override=(0.2, 0.2, 0.2, 0.5),
    root_node_name="/robots/b_base",
)

# Initialize both robots to home position (8-DoF: 6 joints + 2 gripper)
home_config = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
robot_a.update_cfg(home_config)
robot_b.update_cfg(home_config)

print("✓ Robots loaded and initialized")


✓ Robots loaded and initialized


## 4. Initialize MPlib Planner

Set up the motion planning library for IK/FK calculations.


In [52]:
# Initialize MPlib planner
planner = mplib.Planner(
    urdf=str(URDF_PATH),
    srdf=str(SRDF_PATH) if SRDF_PATH.exists() else None,  # Optional SRDF for collision checking
    move_group="ee_gripper_link",  # End-effector link name
)

print("✓ MPlib planner initialized")


✓ MPlib planner initialized


## 5. Helper Functions: Forward and Inverse Kinematics

Utility functions for easy IK/FK calculations.


In [53]:
EE_LINK = "ee_gripper_link"  # End-effector link name

def ee_fk(q, ee=EE_LINK):
    """
    Forward kinematics: joint angles -> end-effector pose.
    
    Args:
        q: Joint angles (can be move-group size or full DoF)
        ee: End-effector link name
        
    Returns:
        position: (3,) array [x, y, z]
        quaternion: (4,) array [w, x, y, z]
    """
    pin = planner.robot.get_pinocchio_model()
    q_full = planner.pad_move_group_qpos(np.asarray(q, float))  # Pad to full DoF if needed
    pin.compute_forward_kinematics(q_full)
    ee_idx = pin.get_link_names().index(ee)
    pose = pin.get_link_pose(ee_idx)
    return np.asarray(pose.get_p(), float), np.asarray(pose.get_q(), float)


# ============================================================================
# COMMENTED OUT IK FUNCTIONS (Alternative implementations)
# ============================================================================

# def ee_ik(pos, quat=None, start=None, tol=1e-2, ee=EE_LINK):
#     """
#     Inverse kinematics (best-effort):
#     - Tries Planner.IK (exact if reachable, else approximate if tolerance allows)
#     - Falls back to CLIK if no IK solution found
    
#     Args:
#         pos: Target position [x, y, z]
#         quat: Target quaternion [w, x, y, z] (optional, uses current orientation if None)
#         start: Starting joint configuration (optional)
#         tol: Position tolerance
#         ee: End-effector link name
        
#     Returns:
#         Full qpos suitable for viewers/FK
#     """
#     pos = np.asarray(pos, float).tolist()
    
#     # Use current configuration if start not provided
#     if start is None:
#         start = planner.robot.get_qpos()
    
#     # Use current orientation if quaternion not provided (position-priority IK)
#     if quat is None:
#         _, quat_now = ee_fk(start, ee)
#         quat = quat_now
#     quat = np.asarray(quat, float).tolist()

#     goal = mplib.Pose(pos, quat)

#     # Try Planner.IK (handles different API versions)
#     try:
#         status, q_goals = planner.IK(
#             goal_pose=goal, 
#             start_qpos=np.asarray(start, float),
#             threshold=tol, 
#             return_closest=True
#         )
#     except TypeError:
#         # Older API (no return_closest)
#         status, q_goals = planner.IK(
#             goal_pose=goal, 
#             start_qpos=np.asarray(start, float),
#             threshold=tol
#         )

#     q_sol_group = None
#     if status == "Success" and q_goals is not None:
#         q_sol_group = (q_goals if isinstance(q_goals, np.ndarray) else q_goals[0])

#     if q_sol_group is not None:
#         return planner.pad_move_group_qpos(q_sol_group)  # Return full DoF

#     # Fallback: CLIK (best-effort closest pose, even if unreachable)
#     pin = planner.robot.get_pinocchio_model()
#     ee_idx = pin.get_link_names().index(ee)
#     q_full0 = planner.pad_move_group_qpos(np.asarray(start, float))
#     q_full_best, _, _ = pin.compute_IK_CLIK(ee_idx, goal, q_full0)
#     return q_full_best

# # print("✓ Helper functions defined")

# ============================================================================
# ACTIVE IK FUNCTION
# ============================================================================

# GRIPPER_SLICE = slice(-2, None)  # last 2 joints

# def ee_ik(pos, quat=None, start=None, tol=1e-2, ee=EE_LINK):
#     # ---- normalize inputs ----
#     pos_arr = np.asarray(pos, float).reshape(3)

#     if start is None:
#         start_full = np.asarray(planner.robot.get_qpos(), float).reshape(-1)
#     else:
#         start_full = np.asarray(start, float).reshape(-1)

#     # cache gripper so we can put it back later
#     gripper_start = start_full[GRIPPER_SLICE].copy()

#     # orientation
#     if quat is None:
#         _, quat_now = ee_fk(start_full, ee)
#         quat_arr = np.asarray(quat_now, float).reshape(4)
#     else:
#         quat_arr = np.asarray(quat, float).reshape(4)

#     goal = mplib.Pose(pos_arr.tolist(), quat_arr.tolist() if quat_arr is not None else None)

#     # Optional: still pass a mask for arm vs gripper (won't hurt)
#     mask = np.zeros_like(start_full, dtype=bool)
#     mask[GRIPPER_SLICE] = True  # don't use gripper in IK

#     status, q_goals = planner.IK(
#         goal_pose=goal,
#         start_qpos=start_full,
#         mask=mask,
#         threshold=tol,
#         return_closest=True,
#     )

#     if q_goals is not None:
#         # handle ndarray vs list-of-arrays
#         if isinstance(q_goals, np.ndarray):
#             q_sol_group = q_goals.reshape(-1)
#         else:
#             q_sol_group = np.asarray(q_goals[0], float).reshape(-1)

#         # expand arm solution to full qpos
#         q_full = planner.pad_move_group_qpos(q_sol_group).reshape(-1)

#         # restore gripper
#         q_full[GRIPPER_SLICE] = gripper_start

#         # ---- SELF-COLLISION CHECK (fast IK path) ----
#         # planner.check_for_self_collision returns a list; empty => no collision
#         collisions = planner.check_for_self_collision(q_full)
#         if not collisions:
#             return q_full  # collision-free IK result

#         # otherwise fall through to best-guess method

#     # 2) Fallback: multi-start CLIK for "closest" pose, then re-clamp gripper
#     q_closest = ee_ik_closest_clik(
#         pos=pos,
#         quat=quat,
#         start=start_full,
#         ee=ee,
#     ).reshape(-1)

#     q_closest[GRIPPER_SLICE] = gripper_start
#     return q_closest


# def ee_ik_closest_clik(
#     pos=None,
#     quat=None,
#     start=None,
#     ee=EE_LINK,
#     n_seeds=16,
#     eps=1e-5,
#     max_iter=800,
#     dt=0.05,
#     damp=1e-4,
# ):
#     """
#     Best-effort IK using multi-start CLIK + custom error.

#     Gripper joints in GRIPPER_SLICE are kept fixed to their value in `start`.
#     Uses planner.check_for_self_collision to discard self-colliding candidates.
#     """
#     print("SWITCHING TO BEST GUESS METHOD")

#     # --- normalize start ---
#     if start is None:
#         start_full = np.asarray(planner.robot.get_qpos(), dtype=np.float64).reshape(-1, 1)
#     else:
#         start_full = np.asarray(start, dtype=np.float64).reshape(-1, 1)

#     # cache gripper from start
#     gripper_start = start_full[GRIPPER_SLICE, 0].copy()

#     if pos is None and quat is None:
#         return start_full.reshape(-1)

#     pin = planner.robot.get_pinocchio_model()
#     ee_idx = pin.get_link_names().index(ee)

#     # --- normalize targets ---
#     pos_target = None
#     quat_target = None

#     if pos is not None:
#         pos_target = np.asarray(pos, dtype=np.float64).reshape(3)

#     if quat is not None:
#         quat_target = np.asarray(quat, dtype=np.float64).reshape(4)
#         quat_target /= np.linalg.norm(quat_target)

#     # --- what "closest" means ---
#     def pose_error(q_full_2d):
#         q_full_2d = np.asarray(q_full_2d, dtype=np.float64).reshape(-1, 1)

#         pin.compute_forward_kinematics(q_full_2d)
#         ee_pose = pin.get_link_pose(ee_idx)
#         p = np.asarray(ee_pose.get_p(), dtype=np.float64).reshape(3)
#         q = np.asarray(ee_pose.get_q(), dtype=np.float64).reshape(4)
#         q /= np.linalg.norm(q)

#         err = 0.0
#         if pos_target is not None:
#             err += np.linalg.norm(p - pos_target)
#         if quat_target is not None:
#             dot = np.abs(np.dot(q, quat_target))
#             dq = 2.0 * np.arccos(np.clip(dot, -1.0, 1.0))
#             err += dq
#         return float(err)

#     # --- seeds: start + random configs (but gripper fixed to start) ---
#     seeds = [start_full.copy()]
#     for _ in range(n_seeds - 1):
#         q_rand = np.asarray(pin.get_random_configuration(), dtype=np.float64).reshape(-1, 1)
#         # force gripper to start value even in seeds
#         q_rand[GRIPPER_SLICE, 0] = gripper_start
#         seeds.append(q_rand)

#     # joint-level mask: True = joint NOT used in IK (optional)
#     n_dof = start_full.shape[0]
#     joint_mask = [False] * n_dof
#     for idx in range(*GRIPPER_SLICE.indices(n_dof)):
#         joint_mask[idx] = True

#     best_q = None
#     best_err = np.inf

#     for q0 in seeds:
#         q0 = np.asarray(q0, dtype=np.float64).reshape(-1, 1)

#         # Seed pose
#         pin.compute_forward_kinematics(q0)
#         ee_pose_seed = pin.get_link_pose(ee_idx)
#         p_seed = np.asarray(ee_pose_seed.get_p(), dtype=np.float64).reshape(3)
#         q_seed = np.asarray(ee_pose_seed.get_q(), dtype=np.float64).reshape(4)
#         q_seed /= np.linalg.norm(q_seed)

#         p_goal = pos_target if pos_target is not None else p_seed
#         q_goal = quat_target if quat_target is not None else q_seed

#         goal_pose = mplib.Pose(p_goal, q_goal)

#         # CLIK: joints with True in mask won't be used in IK (stays at q0)
#         q_candidate, _, _ = pin.compute_IK_CLIK(
#             ee_idx,
#             goal_pose,
#             q0,
#             joint_mask,
#             eps,
#             max_iter,
#             dt,
#             damp,
#         )

#         q_candidate = np.asarray(q_candidate, dtype=np.float64).reshape(-1, 1)

#         # hard clamp gripper back to start
#         q_candidate[GRIPPER_SLICE, 0] = gripper_start

#         # ---- SELF-COLLISION FILTER (CLIK candidates) ----
#         q_flat = q_candidate.reshape(-1)
#         if planner.check_for_self_collision(q_flat):
#             # skip self-colliding candidate
#             continue

#         err = pose_error(q_candidate)
#         if err < best_err:
#             best_err = err
#             best_q = q_candidate.copy()

#     # If everything collided, just keep the start configuration
#     if best_q is None:
#         return start_full.reshape(-1)

#     return best_q.reshape(-1)



GRIPPER_SLICE = slice(-2, None)  # last 2 joints

def ee_ik(
    pos,
    quat=None,
    start=None,
    tol=1e-2,
    ee=EE_LINK,
    n_steps=30,
):
    """
    Best-effort IK along a straight line in workspace:

    1) Get current EE pose (p0, q0) from `start`.
    2) Linearly interpolate positions from p0 -> target pos.
    3) For each intermediate target, run MPLib IK from the previous best config.
    4) Keep the last (closest) pose that:
         - has a valid IK solution, and
         - is self-collision-free.
    5) Gripper joints (GRIPPER_SLICE) are kept fixed to their `start` value.

    If nothing along the line is valid, returns `start` unchanged.
    """
    # ---- normalize start config (full qpos) ----
    if start is None:
        start_full = np.asarray(planner.robot.get_qpos(), float).reshape(-1)
    else:
        start_full = np.asarray(start, float).reshape(-1)

    # cache gripper so we can put it back later
    gripper_start = start_full[GRIPPER_SLICE].copy()

    # ---- current EE pose ----
    p0, q0 = ee_fk(start_full, ee)
    p0 = np.asarray(p0, float).reshape(3)

    # ---- target pose ----
    p_target = np.asarray(pos, float).reshape(3)
    if quat is None:
        quat_arr = None  # keep current orientation
    else:
        quat_arr = np.asarray(quat, float).reshape(4)

    # if the target equals current position, just return start
    if np.allclose(p0, p_target):
        return start_full

    # mask: freeze gripper in IK solve
    mask = np.zeros_like(start_full, dtype=bool)
    mask[GRIPPER_SLICE] = True

    # initialize search
    best_q_full = start_full.copy()
    best_t = 0.0
    q_seed = start_full.copy()   # we keep tracking from the last good solution

    # ---- line search in task space ----
    for i in range(1, n_steps + 1):
        t = i / float(n_steps)
        p_interp = p0 + t * (p_target - p0)

        goal = mplib.Pose(p_interp.tolist(), quat_arr.tolist() if quat_arr is not None else None)

        # IK from previous best config
        status, q_goals = planner.IK(
            goal_pose=goal,
            start_qpos=q_seed,
            mask=mask,
            threshold=tol,
            return_closest=True,
        )
        if q_goals is None:
            # nothing usable at this step, skip
            continue

        # move-group qpos → 1D array
        if isinstance(q_goals, np.ndarray):
            q_sol_group = q_goals.reshape(-1)
        else:
            q_sol_group = np.asarray(q_goals[0], float).reshape(-1)

        # expand to full qpos
        q_full = planner.pad_move_group_qpos(q_sol_group).reshape(-1)

        # restore gripper joints
        q_full[GRIPPER_SLICE] = gripper_start

        # self-collision check
        if planner.check_for_self_collision(q_full):
            # colliding at this interpolation point, skip
            continue

        # this is a valid, collision-free solution closer along the line
        best_q_full = q_full
        best_t = t
        q_seed = q_full.copy()  # next IK starts from here

    # If we never found anything better than t=0, this is just start_full.
    # Otherwise it's the closest collision-free point along the straight line.
    return best_q_full


In [54]:
# Example joint angles (in degrees, converted to radians)
joint_angles_deg = [-38.28, 23.20, 7.99, 22.87, -33.76, 26.68]
joint_angles_rad = np.radians(joint_angles_deg)

# Compute forward kinematics
position, quaternion = ee_fk(joint_angles_rad)

print(f"Joint angles (deg): {joint_angles_deg}")
print(f"End-effector position: {position}")
print(f"End-effector quaternion (w,x,y,z): {quaternion}")

# Update robot visualization
robot_a.update_cfg(np.append(joint_angles_rad, [0.0, 0.0]))  # Add gripper DoF

# Visualize end-effector position (green sphere)
server.scene.add_icosphere(
    "/end_effector",
    radius=0.015,
    color=(0.0, 1.0, 0.0),  # Green
    position=(position[0], position[1] + Y_OFFSET, position[2]),
)


Joint angles (deg): [-38.28, 23.2, 7.99, 22.87, -33.76, 26.68]
End-effector position: [ 0.25718745 -0.07345394  0.230426  ]
End-effector quaternion (w,x,y,z): [ 0.97915298  0.19181383 -0.06164625 -0.02581921]


IcosphereHandle(radius=0.015, subdivisions=3, color=(0, 255, 0), wireframe=False, opacity=None, flat_shading=False, side='front', material='standard', cast_shadow=True, receive_shadow=True)

## 7. Example: Inverse Kinematics

Find joint angles to reach a target end-effector pose.


In [62]:
# Set initial configuration
initial_config = np.array([0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0])
robot_a.update_cfg(initial_config)
robot_b.update_cfg(initial_config)

# Define target position
target_position = np.array([0.11360728, 0.0, 0.11602734])
target_quaternion = [1.0, 0.0, 0.0, 0.0]  # Identity quaternion (w, x, y, z)

# Visualize target (red sphere)
server.scene.add_icosphere(
    "/target",
    radius=0.02,
    color=(1.0, 0.0, 0.0),  # Red
    position=(target_position[0], target_position[1] + Y_OFFSET, target_position[2]),
)
print(f"✓ Target position: {target_position}")

# Solve inverse kinematics
q_solution = ee_ik(
    pos=target_position,
    quat=target_quaternion,
    start=initial_config
)

# Update robot to IK solution
robot_a.update_cfg(q_solution)
print(f"✓ IK solution: {q_solution[:6]} (first 6 joints)")

# Verify with forward kinematics
actual_position, _ = ee_fk(q_solution[:6])
error = np.linalg.norm(actual_position - target_position)
print(f"✓ Position error: {error*1000:.2f} mm")

# Visualize actual end-effector position (green sphere)
server.scene.add_icosphere(
    "/end_effector_ik",
    radius=0.015,
    color=(0.0, 1.0, 0.0),  # Green
    position=(actual_position[0], actual_position[1] + Y_OFFSET, actual_position[2]),
)


✓ Target position: [0.11360728 0.         0.11602734]
✓ IK solution: [ 0.          0.02266828  0.28995426 -0.26728598  0.          0.        ] (first 6 joints)
✓ Position error: 162.39 mm


IcosphereHandle(radius=0.015, subdivisions=3, color=(0, 255, 0), wireframe=False, opacity=None, flat_shading=False, side='front', material='standard', cast_shadow=True, receive_shadow=True)

## 8. IK Examples: Reachable vs Unreachable Positions

Demonstrate IK with a reachable position (exact solution) and an unreachable position (CLIK fallback).


In [68]:
# Set initial configuration
import time
start_config = np.array([0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0])
robot_a.update_cfg(start_config)

print("=" * 60)
print("Example 1: IK with REACHABLE position")
print("=" * 60)

# Reachable target position (within workspace)
reachable_target = np.array([0.4, 0.25, 0.3])
target_quat = [1.0, 0.0, 0.0, 0.0]

# Visualize target (red sphere)
server.scene.add_icosphere(
    "/target_reachable",
    radius=0.02,
    color=(1.0, 0.0, 0.0),  # Red
    position=(reachable_target[0], reachable_target[1] + Y_OFFSET, reachable_target[2]),
)

# Solve IK
q_reachable = ee_ik(pos=reachable_target, quat=target_quat, start=start_config)
robot_a.update_cfg(q_reachable)

# Verify with FK
actual_pos, _ = ee_fk(q_reachable[:6])
error = np.linalg.norm(actual_pos - reachable_target)

print(f"Target: {reachable_target}")
print(f"Actual: {actual_pos}")
print(f"Position error: {error*1000:.2f} mm")
print("✓ IK solution found (exact or approximate within tolerance)")

# Visualize actual end-effector (green sphere)
server.scene.add_icosphere(
    "/ee_reachable",
    radius=0.015,
    color=(0.0, 1.0, 0.0),  # Green
    position=(actual_pos[0], actual_pos[1] + Y_OFFSET, actual_pos[2]),
)

print("\n" + "=" * 60)
print("Example 2: IK with UNREACHABLE position (CLIK fallback)")
print("=" * 60)

# Unreachable target position (outside workspace)
# unreachable_target = np.array([1.0, 0.0, 1.0])  # Too far from base

# 1. Define how far you want the target to be
distance = 1.0

# 2. Create a random vector from a normal distribution.
#    This gives us a random direction in 3D space.
random_direction = np.random.randn(3)

# 3. Normalize the vector to make its length exactly 1.0
#    We do this by dividing it by its own magnitude (the L2 norm).
norm = np.linalg.norm(random_direction)
unit_vector = random_direction / norm

# 4. Scale the unit vector by the desired distance
unreachable_target = unit_vector * distance


# Visualize target (orange sphere to indicate unreachable)
server.scene.add_icosphere(
    "/target_unreachable",
    radius=0.02,
    color=(1.0, 0.5, 0.0),  # Orange
    position=(unreachable_target[0], unreachable_target[1]  + Y_OFFSET, unreachable_target[2]),
)

# Try IK - will fall back to CLIK if unreachable
q_unreachable = ee_ik(pos=unreachable_target, quat=target_quat, start=start_config)
robot_a.update_cfg(q_unreachable)

# Verify with FK
actual_pos_unreachable, _ = ee_fk(q_unreachable[:6])
error_unreachable = np.linalg.norm(actual_pos_unreachable - unreachable_target)

print(f"Target: {unreachable_target}")
print(f"Actual: {actual_pos_unreachable}")
print(f"Position error: {error_unreachable*1000:.2f} mm")
print("⚠ CLIK used (best-effort solution for unreachable target)")

# Visualize actual end-effector (yellow sphere to indicate approximation)
server.scene.add_icosphere(
    "/ee_unreachable",
    radius=0.015,
    color=(1.0, 1.0, 0.0),  # Yellow
    position=(actual_pos_unreachable[0], actual_pos_unreachable[1] + Y_OFFSET, actual_pos_unreachable[2]),
)

print("\n" + "=" * 60)
print("Summary:")
print("  - Red sphere: Reachable target")
print("  - Orange sphere: Unreachable target")
print("  - Green sphere: Actual EE position (reachable case)")
print("  - Yellow sphere: Actual EE position (unreachable case, CLIK)")
print("=" * 60)



Example 1: IK with REACHABLE position
Target: [0.4  0.25 0.3 ]
Actual: [0.39999589 0.24999115 0.29999963]
Position error: 0.01 mm
✓ IK solution found (exact or approximate within tolerance)

Example 2: IK with UNREACHABLE position (CLIK fallback)
Target: [ 0.93336405 -0.29750326 -0.2008068 ]
Actual: [ 0.69526324 -0.18841539 -0.02081378]
Position error: 317.79 mm
⚠ CLIK used (best-effort solution for unreachable target)

Summary:
  - Red sphere: Reachable target
  - Orange sphere: Unreachable target
  - Green sphere: Actual EE position (reachable case)
  - Yellow sphere: Actual EE position (unreachable case, CLIK)


## 9. Example: Visualize Trajectory from Data

Load trajectory data and visualize the end-effector path. 

**Note:** This example requires a parquet file with trajectory data. Uncomment and adjust the file path as needed.


In [57]:
# Uncomment and adjust the path to your trajectory data file
# import pandas as pd
# 
# TRAJECTORY_FILE = "path/to/your/episode_000000.parquet"
# df = pd.read_parquet(TRAJECTORY_FILE)
# 
# # Extract trajectory points using forward kinematics
# trajectory_points = []
# for idx in range(df.shape[0]):
#     angles = df["observation.state"][idx]
#     # Assuming first 6 values are joint angles in degrees
#     joint_angles = np.radians(angles[:6])
#     position, _ = ee_fk(joint_angles)
#     # Apply Y offset for visualization (robot A)
#     trajectory_points.append([position[0], position[1] + Y_OFFSET, position[2]])
# 
# # Visualize trajectory as spline segments
# for i in range(len(trajectory_points) - 1):
#     server.scene.add_spline_catmull_rom(
#         f"traj_segment_{i}",
#         positions=np.array([trajectory_points[i], trajectory_points[i+1]]),
#         color=(0.5, 0.5, 1.0),  # Light blue
#         segments=2,
#     )
# 
# print(f"✓ Visualized {len(trajectory_points)} trajectory points")

print("Trajectory visualization example (commented out - uncomment and provide data file)")


Trajectory visualization example (commented out - uncomment and provide data file)


## 10. Example: Animate Robot from Trajectory Data

Animate robots using trajectory data from a file.

**Note:** Uncomment and adjust the file path as needed.


In [58]:
# Uncomment and adjust the path to your trajectory data file
# import pandas as pd
# 
# TRAJECTORY_FILE = "path/to/your/episode_000000.parquet"
# df = pd.read_parquet(TRAJECTORY_FILE)
# 
# # Animate robots through trajectory
# FRAME_RATE = 20  # frames per second
# 
# for idx in range(df.shape[0]):
#     angles = df["observation.state"][idx]
#     
#     # Extract joint angles for robot A (first 6 joints)
#     joint_angles_a = np.radians(angles[:6])
#     # Extract gripper values (adjust indices based on your data format)
#     gripper_a = (angles[6] / 2) / 10000  # Example conversion
#     config_a = np.append(joint_angles_a, [gripper_a, gripper_a])
#     
#     # Extract joint angles for robot B (next 6 joints)
#     joint_angles_b = np.radians(angles[7:13])
#     gripper_b = (angles[13] / 2) / 10000
#     config_b = np.append(joint_angles_b, [gripper_b, gripper_b])
#     
#     # Update robot configurations
#     robot_a.update_cfg(config_a)
#     robot_b.update_cfg(config_b)
#     
#     time.sleep(1.0 / FRAME_RATE)
# 
# print("✓ Animation complete")

print("Animation example (commented out - uncomment and provide data file)")


Animation example (commented out - uncomment and provide data file)


(viser) Connection closed (1, 0 total)

(viser) Connection opened (2, 1 total), 410 persistent messages

(viser) Connection closed (2, 0 total)

(viser) Connection opened (0, 1 total), 398 persistent messages

(viser) Connection closed (0, 0 total)

(viser) Connection opened (1, 1 total), 398 persistent messages